In [1]:
import umap
import umap.plot
from bokeh.plotting import show, output_notebook
from sklearn import preprocessing
import hdbscan

In [2]:
print(umap.__version__)

0.4.6


In [3]:
import numpy as np
import pandas as pd
import seaborn as sns

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import Window
from pyspark.sql.types import *
import pyspark.sql.functions as F

# Table of contents
1. [Intro to UMAP](#intro)
2. [Data preparation with Pyspark and Pandas](#prep)
3. [Applying UMAP](#umap)
3. [Applying HDBSCAN](#hdb)  
4. [Visualizations](#vis)
5. [Finding medoids of the clusters](#med)
7. [Conclusion](#con)

# Quick intro to the UMAP model <a name="intro"></a>

Uniform Manifold Approximation and Projection (UMAP) is a manifold learning model for dimension reduction and visualization. Below is a quick summary of what the model try to achieve, i.e. the objective function, refer to the <a href="https://arxiv.org/abs/1802.03426">original paper</a> for more details.

The UMAP model seeks to find a low-dimensional representation of the data that has as similar a fuzzy topological structure of the data as possible. It does so by minimizing the fuzzy set cross entropy between the fuzzy topological representation of the data and that of the low-dimensional representation. 

The fuzzy set cross entropy is defined as follows:

\begin{equation}
\sum_{i \neq j}v_{ij}\log(\frac{v_{ij}}{w_{ij}}) + (1-v_{ij})\log(\frac{1-v_{ij}}{1-w_{ij}})
\end{equation} 
We could think of $v_{ij}$ as the similarity between points $i$ and $j$ in the original space (in practice, this would be the weight of the edge between vertices $i$ and $j$ in a weighted graph), and $w_{ij}$ as the similarity between points $i$ and $j$ in the low-dimensional space. 

For a point $i$ in the original space, its similarity to another point $j$ is defined as:

\begin{equation}
v_{j|i} = \exp(\frac{-\max(0,d(x_i,x_j)-\rho_i)}{\sigma_i})
\end{equation} 

where $d$ is a user-defined distance metric, $\rho_i$ is the distance to the $k$th nearest neighbor of point $i$, and $\sigma_i$ is a normalizing factor. Symmetrization is carried out by defining:
\begin{equation}
v_{ij} = (v_{j|i} + v_{i|j}) - v_{j|i}v_{i|j}
\end{equation} 


For a point $i$ in the low-dimensional space, its similarity to another point $j$ is given by:

\begin{equation}
w_{ij} = (1 + a|| y_i - y_j||_{2}^{2b})^{-1}
\end{equation} 
where $a$ and $b$ are hyperparameters.

# Data preparation using PySpark and Pandas <a name="prep"></a>

The raw data we are going to use is MLB's pitch-by-pitch data scraped from the MLB website <a href="https://baseballsavant.mlb.com/">Savant</a>. The parquet file in this repository includes all 3 million pitches from 2014 to 2019. The csv documentation could be found here: https://baseballsavant.mlb.com/csv-docs. We are going to process the data so that we end up with a pitcher arsenal dataframe, with each row representing a pitcher and each column representing a characteristic of the pitcher's pitch (i.e, speed/movement for a certain pitch type).

### Data Cleaning

In [5]:
sc = SparkSession.builder.getOrCreate()

In [6]:
baseball = sc.read.parquet('./baseball_savant.parquet')

In [7]:
baseball = baseball.withColumn('pitcher_team',F.when(baseball.inning_topbot == 'Bot', baseball.away_team).otherwise(baseball.home_team))
baseball = baseball.withColumn('batter_team',F.when(baseball.inning_topbot == 'Top', baseball.away_team).otherwise(baseball.home_team))
baseball = baseball.withColumn('game_date',baseball.game_date.cast(DateType()))
baseball = baseball.withColumn('game_year',F.year(baseball.game_date))

window = Window.partitionBy('pitcher','game_year')
baseball = baseball.withColumn('season_total_pitches',F.count('*').over(window))

In [8]:
baseball = baseball.filter(baseball.pitch_type.isNotNull())

In [9]:
baseball.groupby('pitch_type').count().show()

+----------+-------+
|pitch_type|  count|
+----------+-------+
|        FT| 401239|
|        SC|    113|
|        SL| 583237|
|        FC| 202561|
|        EP|    867|
|        FF|1296645|
|        FS|  54809|
|        PO|    630|
|        KC|  89087|
|        IN|   6390|
|        CH| 378308|
|        CU| 299183|
|        FO|    845|
|        UN|     20|
|        KN|  11453|
|        FA|     10|
|        SI| 304441|
+----------+-------+



In [10]:
# keeping pitches that are more common
valid_pitch_type = ['CH','CU','FS','KC','SL','SI','FF','FC','FT']
baseball = baseball.filter(baseball.pitch_type.isin(valid_pitch_type))

### Create arsenal data on which we wish to run UMAP on

Here we are interested in pitchers that pitched in the 2019 season with more than 200 pitches. The spark code produces the average release speed, horizontal movement, vertical movement, and proportion of each pitch type for each pitcher. At the end, we will generate a 36-dimensional dataframe that will be fed into UMAP.

In [11]:
arsenal = baseball.select('pitch_type','game_year','player_name','pitcher','p_throws','pitcher_team','season_total_pitches','pitcher','pfx_x','pfx_z','release_speed')

In [12]:
arsenal = arsenal.filter('game_year = 2019 and season_total_pitches > 200').groupBy('pitcher','pitch_type','pitcher_team').\
agg(F.first('player_name').alias('player_name'),
    F.first('p_throws').alias('p_throws'),
    F.avg('release_speed').alias('avg_speed'), 
    F.avg('pfx_z').alias('avg_z'),
    F.avg('pfx_x').alias('avg_x'),
    F.count('*').alias('count'))

In [13]:
window = Window.partitionBy('pitcher','pitcher_team')
arsenal = arsenal.withColumn('proportion', F.col('count')/F.sum('count').over(window))

In [14]:
# convert from a spark dataframe to a Pandas dataframe
arsenal = arsenal.toPandas()

In [15]:
# data for Masahiro Tanaka (long format, we will transform it to wide format later)
arsenal[arsenal.player_name == 'Masahiro Tanaka']

,pitcher,pitch_type,pitcher_team,player_name,p_throws,avg_speed,avg_z,avg_x,count,proportion
2936,547888,SI,NYY,Masahiro Tanaka,R,90.335294,0.725872,-1.235809,136,0.045546
2937,547888,SL,NYY,Masahiro Tanaka,R,83.248861,0.108844,0.548934,1097,0.367381
2938,547888,FC,NYY,Masahiro Tanaka,R,87.460870,0.672999,-0.141720,46,0.015405
2939,547888,FS,NYY,Masahiro Tanaka,R,86.724876,0.428213,-1.064891,808,0.270596
2940,547888,CU,NYY,Masahiro Tanaka,R,76.069231,-0.750867,0.770970,91,0.030476
2941,547888,FF,NYY,Masahiro Tanaka,R,91.439851,1.334656,-0.840529,808,0.270596


In [16]:
# sometimes the MLB classify pitches incorrectly, we want to get rid of those records
arsenal2 = arsenal[arsenal.proportion > 0.01]

In [17]:
# transform the data so that each row now represents a pitcher (wide format)
df = arsenal2.pivot_table(index = ['pitcher','player_name','p_throws','pitcher_team'], columns = 'pitch_type', values = ['avg_x','avg_z','avg_speed','proportion'])
df.columns = [x[0] + '_' + x[1] for x in df.columns]
df.reset_index(inplace = True)
df.fillna(0,inplace = True)

In [18]:
pd.set_option('display.max_columns', None)

In [19]:
df.sample(5)

,pitcher,player_name,p_throws,pitcher_team,avg_speed_CH,avg_speed_CU,avg_speed_FC,avg_speed_FF,avg_speed_FS,avg_speed_FT,avg_speed_KC,avg_speed_SI,avg_speed_SL,avg_x_CH,avg_x_CU,avg_x_FC,avg_x_FF,avg_x_FS,avg_x_FT,avg_x_KC,avg_x_SI,avg_x_SL,avg_z_CH,avg_z_CU,avg_z_FC,avg_z_FF,avg_z_FS,avg_z_FT,avg_z_KC,avg_z_SI,avg_z_SL,proportion_CH,proportion_CU,proportion_FC,proportion_FF,proportion_FS,proportion_FT,proportion_KC,proportion_SI,proportion_SL
93,489265,Sergio Romo,R,MIA,79.940426,0.000000,0.000000,86.614035,0.0,0.000000,0.0,86.270833,77.050000,-1.696721,0.000000,0.0000,-1.174678,0.0,0.000000,0.0,-1.613596,1.327461,0.202187,0.000000,0.000000,0.987733,0.0,0.000000,0.0,0.246074,0.698516,0.160684,0.000000,0.000000,0.097436,0.0,0.000000,0.0,0.164103,0.577778
709,666200,Jesus Luzardo,L,OAK,86.671429,82.761765,0.000000,97.000000,0.0,95.984906,0.0,0.000000,83.925000,1.461906,-0.463782,0.0000,0.906752,0.0,1.385182,0.0,0.000000,-0.559348,0.454620,-0.380091,0.000000,1.311936,0.0,0.833569,0.0,0.000000,-0.023164,0.193548,0.313364,0.000000,0.230415,0.0,0.244240,0.0,0.000000,0.018433
728,676606,Nick Margevicius,L,SD,79.001449,70.806202,0.000000,88.255985,0.0,0.000000,0.0,0.000000,80.186777,0.349070,-0.612754,0.0000,0.234903,0.0,0.000000,0.0,0.000000,-0.312517,0.970261,-0.942318,0.000000,1.433777,0.0,0.000000,0.0,0.000000,0.092926,0.072025,0.134656,0.000000,0.540710,0.0,0.000000,0.0,0.000000,0.252610
162,520980,Pedro Baez,R,LAD,86.211111,0.000000,0.000000,95.829072,0.0,0.000000,0.0,0.000000,86.375385,-0.847976,0.000000,0.0000,-0.451751,0.0,0.000000,0.0,0.000000,0.233158,1.252712,0.000000,0.000000,1.508864,0.0,0.000000,0.0,0.000000,0.385743,0.313393,0.000000,0.000000,0.509821,0.0,0.000000,0.0,0.000000,0.174107
73,467100,Ivan Nova,R,CWS,86.191268,79.867063,86.705371,92.458144,0.0,92.303542,0.0,0.000000,0.000000,-1.378018,0.232450,-0.3033,-1.033882,0.0,-1.433255,0.0,0.000000,0.000000,0.610043,-0.521787,0.673412,0.981517,0.0,0.594099,0.0,0.000000,0.000000,0.160067,0.167720,0.130116,0.175707,0.0,0.366389,0.0,0.000000,0.000000


In [20]:
df_umap = df.drop(['pitcher','player_name','p_throws','pitcher_team'],axis = 1)

In [21]:
# Scale each column by its maximum absolute value
cols_to_scale = [col for col in df if col.startswith('avg')]
preprocessor = preprocessing.MaxAbsScaler().fit(df_umap[cols_to_scale])
df_umap[cols_to_scale] = preprocessor.transform(df_umap[cols_to_scale])

In [22]:
# 36-dimensional data that will be fed into the UMAP algorithm.
df_umap.iloc[190]

avg_speed_CH     0.000000
avg_speed_CU     0.928290
avg_speed_FC     0.939670
avg_speed_FF     0.975593
avg_speed_FS     0.953298
avg_speed_FT     0.000000
avg_speed_KC     0.000000
avg_speed_SI     0.000000
avg_speed_SL     0.918602
avg_x_CH         0.000000
avg_x_CU         0.378997
avg_x_FC         0.061649
avg_x_FF        -0.591736
avg_x_FS        -0.489777
avg_x_FT         0.000000
avg_x_KC         0.000000
avg_x_SI         0.000000
avg_x_SL         0.321928
avg_z_CH         0.000000
avg_z_CU        -0.457214
avg_z_FC         0.558999
avg_z_FF         0.633422
avg_z_FS         0.102651
avg_z_FT         0.000000
avg_z_KC         0.000000
avg_z_SI         0.000000
avg_z_SL         0.035081
proportion_CH    0.000000
proportion_CU    0.174842
proportion_FC    0.225475
proportion_FF    0.432753
proportion_FS    0.135285
proportion_FT    0.000000
proportion_KC    0.000000
proportion_SI    0.000000
proportion_SL    0.031646
Name: 190, dtype: float64

# Apply UMAP on pitcher arsenal data<a name="umap"></a>

Here we apply the UMAP algorithm and project the data onto a 2 dimensional space. There are two hyper-parameters: n_neighbors, the number of nearest neighbors when constructing the graph in the original space, and min_dist, the minimum distance between points in the low-dimensional embedding space. <br>

Increasing n_neighbors would let us preserve more the global structure of the data, while decreasing n_neighbors let us focus on the local structure of the data. The second hyper-parameter min_dist is more of a aesthetic hyper-paramter, which determines the minimum distance between points in the embedding space.

In [23]:
# N_components is the dimension that the data is projected onto
N_COMPONENTS = 2
METRIC = 'cosine'
transformer = umap.UMAP(n_components = N_COMPONENTS, random_state = 0,n_neighbors=30,min_dist=0.0, metric = METRIC).fit(df_umap)
embedding = transformer.transform(df_umap)

# Apply HDBSCAN on the UMAP embeddings <a name="hdb"></a>

After learning a low-dimensional representation for the pitcher data, we could apply a clustering algorithm on the embeddings to detect pitcher clusters. In particular, I chose HDBSCAN. HDBSCAN is a hierarchical density-based clustering algorithm. Unlike K-Means, HDBSCAN works well with clusters having varying densities and shapes. The main hyper-parameter we would control is the minimum cluster size, which I set to be 15.

The HDBSCAN gives us 17 clusters:

In [24]:
# Fit HDBSCAN with minimum cluster size 15
clusterer = hdbscan.HDBSCAN(min_cluster_size=15)
clusterer.fit(embedding)
# number of clusters that are detected by HDBSCAN
print('Number of clusters: {}'.format(clusterer.labels_.max()+1))

Number of clusters: 17


In [25]:
embedding_dat = pd.DataFrame(np.column_stack([embedding,clusterer.labels_,df[['player_name','pitcher','p_throws','pitcher_team']]]),
                             columns = ['x{}'.format(i) for i in range(1,N_COMPONENTS+1)]+['cluster','player_name','pitcher','p_throws','team'])\
                            .sort_values('cluster')

In [26]:
embedding_dat.sample(10)

,x1,x2,cluster,player_name,pitcher,p_throws,team
62,-0.515863,10.9933,8,Lance Lynn,458681,R,TEX
97,8.22103,8.04268,10,Jeanmar Gomez,491646,R,TEX
514,2.6261,5.64826,15,Colin Poche,621363,L,TB
98,4.45869,4.27654,13,Seunghwan Oh,493200,R,COL
80,-0.377554,-0.917072,6,Brett Anderson,474463,L,OAK
46,2.46016,8.6442,-1,Mark Melancon,453343,R,SF
315,9.53899,8.97217,2,Adrian Sampson,592716,R,TEX
81,0.980691,10.0584,11,Chaz Roe,475054,R,TB
73,-0.466796,11.0281,8,Ivan Nova,467100,R,CWS
24,1.49696,0.460411,3,Rich Hill,448179,L,LAD


# Visualization of the embeddings <a name="vis"></a>

In [29]:
from bokeh.plotting import show, output_notebook

## Points colored by handedness:
We are able to separate the left-handed and right-handed pitchers pretty well.

In [30]:
p = umap.plot.interactive(transformer, labels=embedding_dat.p_throws, hover_data=embedding_dat[['player_name','cluster','p_throws','team']], theme = 'fire')
output_notebook()
show(p)

Loading BokehJS ...

## Points colored by cluster membership:

In [32]:
# -1 represents points that are viewed as noise by the HDBSCAN
p = umap.plot.interactive(transformer, labels=embedding_dat.cluster, hover_data=embedding_dat[['player_name','cluster','p_throws','team']], theme = 'fire')
output_notebook()
show(p)

Loading BokehJS ...

# Medoids of the clusters <a name="med"></a>
We could find the medoids of the clusters to get a general picture of the clusters' characterisitics. Note that there are 4 left-handed clusters (all in the bottom left corner) and 13 right-handed clusters.

In [30]:
from scipy.spatial import distance_matrix

In [31]:
medoids = []
for i in np.unique(embedding_dat.cluster):
    cluster_i_dat = embedding_dat[embedding_dat.cluster == i]
    distMatrix_i = distance_matrix(cluster_i_dat[['x1','x2']],cluster_i_dat[['x1','x2']])
    medoid_index = np.argmin(distMatrix_i.sum(axis=0))
    medoids.append(cluster_i_dat.iloc[medoid_index])

In [32]:
medoids = pd.DataFrame(medoids).iloc[1:]

In [33]:
medoids

,x1,x2,cluster,player_name,pitcher,p_throws,team
103,-7.568806,1.388957,0,Jose Alvarez,501625,L,PHI
298,11.331825,0.631474,1,Ryan Eades,592280,R,MIN
699,9.425493,9.648911,2,Tanner Anderson,664196,R,OAK
683,1.372253,0.068187,3,Patrick Sandoval,663776,L,LAA
172,3.063716,12.489758,4,Jose Cisnero,542585,R,DET
443,0.620855,0.472470,5,Roenis Elias,606273,L,WSH
0,-0.382506,-0.858753,6,CC Sabathia,282332,L,NYY
621,-1.531546,13.270353,7,Zac Reininger,643617,R,DET
120,-0.477669,11.023055,8,Chase Anderson,502624,R,MIL
636,8.464023,7.247468,9,Sam Coonrod,656322,R,SF


# Conclusion <a name="con"></a>

By applying UMAP, we were able to learn a low-dimensional latent represenation of MLB pitchers. We successfully visualized a 36-dimensional pithcer arsenal data and detected potential pitcher clusters. Utilizing the cluster information, one could investigate the characteristics of different pitcher clusters, or how a batter performs against different types of pitchers.